# Imports

In [2]:
import pandas as pd
import numpy as np
import nltk
# nltk.download('punkt')
from collections import Counter
# import fasttext.util
import os
import re
from collections import Counter
from tqdm.auto import tqdm
tqdm.pandas()
# fasttext.util.download_model('en', if_exists='ignore')

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# TF IDF

In [2]:
# get generic word freqs
# for each doc get word freqs
# calculate top doc word freq / generic word freq

In [6]:
# ! pip install fasttext-wheel

In [5]:
df = pd.read_excel('data/SUBTLEXusfrequencyabove1.xls', index_col=0)
freqs = df['FREQcount']
freqs['the']

1501908

In [20]:
data = []
dirname = 'data/thoughts/Thoughts_en/'
for fname in tqdm(os.listdir(dirname)):
    if not fname.endswith('.docx'):
        continue
    with open(f"{dirname}/{fname}") as f:
        data +=  [f.read()]

In [21]:
sentences = [s for d in data for s in d.split('\n')]

In [29]:
sentences = [s.strip() for s in sentences]

In [30]:
sentences = [re.sub('&#39;', '\'', s) for s in sentences if s]

In [47]:
words = [w for s in sentences for w in nltk.word_tokenize(s.lower())]

In [46]:
# words = [w for s in sentences for w in s.lower().split()]

In [40]:
len(words)

85266

In [48]:
cnt = Counter(words)

In [42]:
tf_idf = {c: n/(freqs[c] if c in freqs else 1) for c,n in cnt.items()}

In [49]:
tf_idf = {c: n/freqs[c] for c,n in cnt.items() if c in freqs}

In [50]:
l = sorted([(c,w) for w,c in tf_idf.items()], reverse=True)

In [51]:
l[:50]

[(12.5, 'asana'),
 (2.5714285714285716, 'shekels'),
 (2.5, 'inputs'),
 (2.5, 'app'),
 (1.5, 'startup'),
 (1.4285714285714286, 'neuron'),
 (1.0, 'stereoscopic'),
 (1.0, 'minimums'),
 (1.0, 'leftists'),
 (1.0, 'forums'),
 (1.0, 'explanatory'),
 (1.0, 'decreases'),
 (1.0, 'computation'),
 (0.8888888888888888, 'hazelnut'),
 (0.8333333333333334, 'transitive'),
 (0.8181818181818182, 'entropy'),
 (0.7777777777777778, 'fractal'),
 (0.7692307692307693, 'emails'),
 (0.75, 'melatonin'),
 (0.75, 'expend'),
 (0.7333333333333333, 'gt'),
 (0.7142857142857143, 'causality'),
 (0.7, 'bookmarks'),
 (0.6666666666666666, 'ui'),
 (0.6666666666666666, 'hippocampus'),
 (0.6666666666666666, 'derivatives'),
 (0.631578947368421, 'reminders'),
 (0.625, 'disadvantages'),
 (0.6, 'hierarchical'),
 (0.6, 'cocain'),
 (0.5, 'wardrobes'),
 (0.5, 'uploads'),
 (0.5, 'unpleasantly'),
 (0.5, 'thirsts'),
 (0.5, 'streamer'),
 (0.5, 'solvable'),
 (0.5, 'radars'),
 (0.5, 'pur'),
 (0.5, 'postpones'),
 (0.5, 'outgrowth'),
 (0.5, 

In [17]:
sum([f for f,w in l])

36.26542819941986

# Sentence Embeddings

In [25]:
# fasttext.util.download_model('en', if_exists='ignore', dimension=100)

In [17]:
# import io

# def load_vectors(fname):
#     fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
#     n, d = map(int, fin.readline().split())
#     data = {}
#     for line in fin:
#         tokens = line.rstrip().split(' ')
#         data[tokens[0]] = map(float, tokens[1:])
#     return data

In [18]:
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip -P data/

In [19]:
# ! unzip data/wiki-news-300d-1M.vec.zip

In [20]:
# ft = load_vectors('data/wiki-news-300d-1M.vec')

In [ ]:
# ! pip install -U sister
# fails due to c++ compiler mismatch

In [7]:
#  ! pip install -U sentence-transformers --no-cache-dir

In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [19]:
sentences = data.split('\n')

In [20]:
sentence_embeddings = model.encode(sentences)

In [27]:
sentence_embeddings = np.ndarray.tolist(sentence_embeddings)

In [111]:
docs = []
embeddings = []
dirname = "data/Thoughts_en"
for doc in tqdm(os.listdir(dirname)):
    if not doc.endswith('.docx'):
        continue
    with open(os.path.join(dirname, doc)) as f:
        data = f.read()
    sentences = data.split('\n')
    sentence_embeddings = model.encode(sentences)
    docs += sentences
    embeddings += np.ndarray.tolist(sentence_embeddings)


In [112]:
embeddings_df = pd.DataFrame(embeddings)

In [146]:
embeddings_df.to_csv('data/Thoughts_en.vec.tsv', index=False, header=False, sep='\t')

In [114]:
docs = [re.sub('\t', '    ', doc) for doc in docs]

In [35]:
with open('data/Thoughts_en.txt.tsv', 'w') as f:
    f.write('\n'.join(docs))

In [116]:
all_docs = '\n'.join(docs)

In [117]:
cnt = Counter(all_docs)

In [118]:
cnt = sorted(cnt.items(), key=lambda x: x[1])

In [120]:
low_cnt = [c for c in cnt if c[1]<=10]

In [121]:
for c in low_cnt:
    all_docs = re.sub(c[0], '_', all_docs)

In [74]:
# i = all_docs.index(cnt[6][0])

In [75]:
# all_docs[i-10:i+10]

In [122]:
docs = all_docs.split('\n')

In [124]:
docs_df = pd.DataFrame(docs)

In [130]:
docs_df['action'] = docs_df[0].apply(lambda x: 'need' in x)

In [139]:
docs_df.columns = ['doc', 'is_action']

In [141]:
docs_df.to_csv('data/Thoughts_en.doc.tsv', index=False, sep='\t')